<div>
<table style="width: 100%">
	<tr>
		<td>
		<table style="width: 100%">
			<tr>
                <td ><center><font size="5"><b>Module 49</b></font><center>
                <center><font size="6">Digital Innovations for Water Challenges</font><center></td>
			</tr>
			<tr>
                <td><center><font size="14">Notebook 2.c</font><center></td>
			</tr>
			<tr>
                <td><center><font size="6"><b>Temporal aggregation from tiff files</b></font><center></td>
			</tr>
		</table>
		</td>
		<td><center><img src='images\ihe-delft-institute_unesco_fc-lr.jpg'></img></td>
	</tr>
</table>
</div>

# Table of contents
1. [Learning objectives](#learningobs)
2. [Introduction](#introduction)
3. [Aggregation from tiff files](#tiff)
4. [Aggregation from one .nc file](#nc)

# 1. Learning objectives<a name="learningobs"></a>

- Use xarray to open .tiff and .nc files
- Cycle through files in a directory
- Plot tiff and .nc files
- Make yearly map from monthly data 

# 2. Introduction<a name="introduction"></a>
In this notebook you will learn how to make basic aggregations of .tiff files by cycling through files in a folder and of .nc files using xarray.
You will need the following packages installed:
> xarray

In [ ]:
import os
import glob
import xarray as xr

### 3. Aggregation from tiff files <a name='tiff'></a>
### 3.a. Opening one tiff file
Open one tiff file from the data you downaload usng notebook 1.a.

In [ ]:
filename = r"./CHIRPS_tiff/chirps-v2.0.2020.01.tif" #modify this line as needed
data = xr.open_rasterio(filename)

Inspect the data: how many rows and columns does your data have? 

In [ ]:
data

#### 3.b. Simple plot 
As the data has only one band, running the line below will produce a map of the data

In [ ]:
data.plot()

### 3.c. Masking out no data values
You will notice the data over land looks washed out - this is because the no data value of -9999 was not correctly identified. To mask out these values and obtain a better map try the following cell.

In [ ]:
ds_masked = data.where(data != -9999.) #this keeps only data for pixels with values different than -9999
ds_masked.plot()

### 3.d Yearly precipitation from monthly
Now we will open all data one after the other and make a map of the yearly sum of precipitation.<br>
#### Step 1: Getting all the file names

In [ ]:
#glob allows you to list all files in a folder. 
#You can also specify what the file name needs to look like for inclusion using the * wildcard as shown here
glob.glob(r'./CHIRPS_tiff/*.tif')

#### Step 2: loop through the filenames to open the monthly data and produce a map of the sum

In [ ]:
for i, filename in enumerate(glob.glob(r'./CHIRPS_tiff/*.tif')):
    data = xr.open_rasterio(filename)
    ds_masked = data.where(data != -9999.)
    if i == 0:
        data_sum = ds_masked
    else:
        data_sum += ds_masked

In [ ]:
data_sum.plot()

# 4. Aggregating from one .nc file <a name='nc'></a>
Using notebook 1.a, download monthly CHIRPS data for the years 2020-2021 for the bounding box of your choice.

In [ ]:
# replace with your own path
path_to_nc = r"./CHIRPS_nc/chirps20GlobalMonthlyP05_ea7a_a7e6_216e.nc"

In [ ]:
data_nc = xr.open_dataset(path_to_nc)

Inspect your data - how many dimensions does this dataset have?

In [ ]:
data_nc

1. Average yearly P map
Produce a map of the average monthly precipitation over the area

In [ ]:
data_mean = data_nc.mean(dim='time')
data_mean.precip.plot()

2. Yearly average and sum
From the monthly data compute and plot a yearly average time series

In [ ]:
yearly_mean = data_nc.groupby('time.year').mean('time')
yearly_sum = data_nc.groupby('time.year').sum('time', skipna=False)

In [ ]:
yearly_sum.precip.loc[{'year':2020}].plot()

Get mean average precipitation for whole area:

In [ ]:
yearly_sum.precip.mean(dim=['latitude', 'longitude'])

3. Montly average    
From the data make a bar plot of average precipitation for the months of the year